In [1]:
# df_adjOHLCV has data only for NYSE trading days, no weekend data
# df_adjOHLCV includes data for weekends when BTC trades
# read symbols in file to list syms_in_file
# download OHLCV data for symbols in syms_in_file
# drop symbols with all NaN in OHLCV columns from df
# rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# drop weekend data by reindex to date index of index_symbol
# pickled df_adjOHLCV
# pickled df_adjOHLCV
# pickled symbols_df_adjOHLCV
# create df_symbols_close, sort df by symbols
# pickled df_symbols_close

In [2]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_symbols = path_data_dump + 'vg_symbols_4chars_max_clean.csv'  # symbols text file

filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

In [3]:
# function to adjust OHLCV
# https://github.com/ranaroussi/yfinance/issues/687
def auto_adjust(data):
    df = data.copy()
    ratio = df["Close"] / df["Adj Close"]
    df["Adj Open"] = df["Open"] / ratio
    df["Adj High"] = df["High"] / ratio
    df["Adj Low"] = df["Low"] / ratio

    df.drop(
        ["Open", "High", "Low", "Close"],
        axis=1, inplace=True)

    df.rename(columns={
        "Adj Open": "Open", "Adj High": "High",
        "Adj Low": "Low", "Adj Close": "Close"
    }, inplace=True)

    # df = df[["Open", "High", "Low", "Close", "Volume"]]
    return df[["Open", "High", "Low", "Close", "Volume"]]

def close_vs_Yahoo (symbols, df):
  for symbol in symbols:
    s = df.iloc[-250]
    
    sDate = s.name.strftime('%Y-%m-%d')
    sClose = s[symbol].Close

    df_sym = yf.Ticker(symbol).history(period='2y')
    # 10:45PM got Adj Close
    yhClose = df_sym.loc[sDate]['Close']

    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_err = f'{symbol}  %_dif_Close > .0001'
      # raise SystemExit(msg_err)
      print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
    if symbol == symbols[-1]:
      msg_done = f"No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close "
      print(msg_done)


In [4]:
# Stop if last date in df_adjOHLCV is the same as Yahoo download
index_symbol = "XOM"  
df_XOM = yf.download(index_symbol)
download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
print(f'download_last_date: {download_last_date}')
df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
df_adjOHLCV_last_date = df.index[-1].strftime('%Y-%m-%d')
print(f'df_adjOHLCV_last_date: {df_adjOHLCV_last_date}')
if download_last_date == df_adjOHLCV_last_date:
  msg_stop = f"df_adjOHLCV is up-to-date,\nlast date from df_adjOHLCV is {df_adjOHLCV_last_date},\nlast date from Yahoo's {index_symbol} download is {download_last_date}"
  raise SystemExit(msg_stop )

[*********************100%***********************]  1 of 1 completed
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
download_last_date: 2023-03-02
df_adjOHLCV_last_date: 2023-03-01


In [5]:
# verify df test_symbols' close against Yahoo
# even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
test_symbols = ['A', 'LMT', 'SPY']
_df_list=[]
_df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)
_df_list.append(_df)
_df = pd.concat(_df_list, axis=1)
# get unique symbols from column name
_l_symbols = list(set(_df.columns.get_level_values(0)))
_l_symbols.sort()

_df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  _df1 = auto_adjust(_df[symbol])
  _df_adj_list.append(_df1)

_df_adj = pd.concat(_df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
_df_adj.columns = _col_names

close_vs_Yahoo(test_symbols, _df_adj)

[*********************100%***********************]  3 of 3 completed
symbol:     A   Date: 2022-03-04   df_Close:  133.01483     Yahoo_Close:  133.01482   %_dif_Close: 0.00001
symbol:   LMT   Date: 2022-03-04   df_Close:  446.68860     Yahoo_Close:  446.68860   %_dif_Close: 0.00000
symbol:   SPY   Date: 2022-03-04   df_Close:  425.27072     Yahoo_Close:  425.27069   %_dif_Close: 0.00001
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [6]:
# read symbols in file to a list and convert to chunks
#  each chunk must have more than 1 symbol, otherwise, symbol won't appear as column name
symbols_in_file = read_symbols_file(filename_symbols)
print(f'symbols in file: {len(symbols_in_file)}')
# TODO add list of SPY, VGT, FTSM, QQQ, BNDX 
symbols_add = ['SPY', 'VGT', 'FTEC', 'QQQ', 'VWO', 'VTV', 'BNDX', 'USO', 'JJC', 'BCI', 'NG']
print(f'symbols added:   {len(symbols_add)}')
symbols = symbols_in_file + symbols_add
# https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python
symbols = list(set(symbols))  # keep unique symbols
symbols.sort()  # sort unique symbols in alphabetical order
print(f'unique symbols:  {len(symbols)}')
symbols_chunks = chunked_list(symbols, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]
for chunk in symbols_chunks:
  if len(chunk) == 1:
    raise SystemExit(f'ERROR, Must be more than 1 symbol in each chunk\nsymbols in chunk: {chunk}')

symbols in file: 1314
symbols added:   11
unique symbols:  1319


In [7]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(symbols_chunks):
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
  symbols_download_err.append(list(shared._ERRORS.keys()))
  # print(f'symbols_download_err: {symbols_download_err}')
  print(f'\nsymbols_download_err: {symbols_download_err}')  
  df_list.append(df)
  # pause between download
  if i < len(symbols_chunks) - 1 :  # skip pause after last download
    print(f'downloaded symbols from chuck {i}, sleep start')
    # sleep 78(18m 25s)
    time.sleep(78)
    print(f'downloaded symbols from chuck {i}, sleep ends')
  else:
    print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

[*********************100%***********************]  400 of 400 completed

symbols_download_err: [[]]
downloaded symbols from chuck 0, sleep start
downloaded symbols from chuck 0, sleep ends
[*********************100%***********************]  400 of 400 completed

symbols_download_err: [[], []]
downloaded symbols from chuck 1, sleep start
downloaded symbols from chuck 1, sleep ends
[*********************100%***********************]  400 of 400 completed

symbols_download_err: [[], [], []]
downloaded symbols from chuck 2, sleep start
downloaded symbols from chuck 2, sleep ends
[*********************100%***********************]  119 of 119 completed

symbols_download_err: [[], [], [], []]
downloaded symbols from all chucks
symbols_download_err: [[], [], [], []]


In [8]:
# flatten symbols_download_err which is a list-of-lists
l_symbols_err = [val for sublist in symbols_download_err for val in sublist]
pickle_dump(l_symbols_err, path_data_dump, 'l_symbols_err')
l_symbols_err

[]

In [9]:
# if l_symbols_err:  # re-download symbols with download error 
#   symbols_download_err = []
#   # took 24 minutes to download 1917 symbols without error caused by Yahoo
#   # for i, symbols in enumerate(symbols_chunks):
#   for i, symbols in enumerate(l_symbols_err):
#     df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
#     symbols_download_err.append(list(shared._ERRORS.keys()))
#     # print(f'symbols_download_err: {symbols_download_err}')
#     print(f'\nsymbols_download_err: {symbols_download_err}')  
#     df_list.append(df)
#     # pause between download
#     if i < len(symbols_chunks) - 1 :  # skip pause after last download
#       print(f'downloaded symbols from chuck {i}, sleep start')
#       # sleep 78(18m 25s)
#       time.sleep(78)
#       print(f'downloaded symbols from chuck {i}, sleep ends')
#     else:
#       print(f'downloaded symbols from all chucks')

#   print(f'symbols_download_err: {symbols_download_err}')

In [10]:
df = pd.concat(df_list, axis=1)
if l_symbols_err:  # if list is not empty, drop symbols with errors
  print(f'l_symbols_err is empty: {l_symbols_err}')  
  df.drop(l_symbols_err, axis=1, level=0, inplace=True)  # drop symbols with errors

l_symbols = list(set(df.columns.get_level_values(0)))  # get unique symbols
l_symbols.sort()
df = df[l_symbols]  # sort columns by list
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
pickle_dump(df, path_data_dump, filename_pickled_df_OHLCVA_downloaded)  # save OHLCA data

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [11]:
# adjust OHLC
df_adj_list=[]
for symbol in l_symbols:
  _df = auto_adjust(df[symbol])
  df_adj_list.append(_df)

df_adj = pd.concat(df_adj_list, axis=1)
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
col_names = pd.MultiIndex.from_product([l_symbols, cols])  # create multilevel column names
df_adj.columns = col_names  # reindex columns

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adj, path_data_dump, filename_pickled_df_adjOHLCV)  # save adjusted OHLCV data

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [12]:
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV)

In [13]:
# drop symbols with all NaN in OHLCV columns from df
df_adjOHLCV, symbols_OHLCV, symbols_dropped = drop_symbols_all_NaN(df_adjOHLCV, verbose)
print(f'symbols with all NaN dropped from df_adjOHLCV: {symbols_dropped}')

Symbol's OHLCV are all NaN: []
symbols with all NaN dropped from df_adjOHLCV: []


In [14]:
# rename columns OHLCV *ONLY AFTER* dropping symbols with all NaN from df,
#   symbols with all NaN has an added AdjClose column and will cause errors  
#  rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
#  .remove_unused_levels() prevents ValueError
#   e.g ValueError: On level 1, code max (5) >= length of level (5). NOTE: this index is in an inconsistent state
# The error may be caused by removing symbols from the dataframe with all NaN in OHLCV columns
df_adjOHLCV.columns = df_adjOHLCV.columns.remove_unused_levels()
# # set_levels reorders df columns in alphabetical order, so the list of column names also needs to be in alphabetical order
# df_adjOHLCV.columns = df_adjOHLCV.columns.set_levels(['close', 'high', 'low', 'open', 'volume'], level=1)

In [15]:
# drop weekend data by re-indexing to date-index of index_symbol
myNaN = float('nan')
# use Exxon's date as proxy for NYSE trading dates
df_adjOHLCV = df_adjOHLCV.reindex(df_XOM.index, fill_value=myNaN)

In [16]:
# pickle df_adjOHLCV and symbols
# print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
# pickle_dump(df_adjOHLCV_downloaded, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adjOHLCV, path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
pickle_dump(symbols_OHLCV, path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [17]:
symbols_OHLCV = list(set([i[0] for i in list(df_adjOHLCV)]))
df_symbols_close = df_adjOHLCV.xs('Close', level=1, axis=1)
pickle_dump(df_symbols_close, path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

In [18]:
# retrieve pickled files

print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA_downloaded = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
symbols_df = pickle_load(path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [19]:
print(f'df_OHLCVA_downloaded.info():\n{df_OHLCVA_downloaded.info()}\n')
print(f'df_adjOHLCV.info():\n{df_adjOHLCV.info()}\n')
print(f'df_close.info():\n{df_close.info()}\n')
print(f'len(symbols_df):\n{len(symbols_df)}\n')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15397 entries, 1962-01-02 to 2023-03-02
Columns: 7914 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(7887), int64(27)
memory usage: 929.8 MB
df_OHLCVA_downloaded.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15397 entries, 1962-01-02 to 2023-03-02
Columns: 6595 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(6568), int64(27)
memory usage: 774.8 MB
df_adjOHLCV.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15397 entries, 1962-01-02 to 2023-03-02
Columns: 1319 entries, A to ZWS
dtypes: float64(1319)
memory usage: 155.1 MB
df_close.info():
None

len(symbols_df):
1319



In [20]:
count_symbols = len(symbols_df)
count_df_close_cols = len(df_close.columns)
count_cols_df_OHLCVA_downloaded = len(df_OHLCVA_downloaded.columns)
count_cols_df_adjOHLCV = len(df_adjOHLCV.columns)
if count_symbols != count_df_close_cols:
  raise SystemExit(f'symbol count: {count_symbols} does not equal df_close column count: {count_df_close_cols}')
if count_symbols * 6 != count_cols_df_OHLCVA_downloaded:
  raise SystemExit(f'6 x symbol count: {6 * count_symbols} does not equal df_OHLCVA_downloaded column count: {count_cols_df_OHLCVA_downloaded}')
if count_symbols * 5 != count_cols_df_adjOHLCV:
  raise SystemExit(f'5 x symbol count: {5 * count_symbols} does not equal df_adjOHLCV column count: {count_cols_df_adjOHLCV}')
print(f'No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count')


No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count


In [21]:
print(f'count_symbols: {count_symbols}')
print(f'count_df_close_cols: {count_df_close_cols}')
print(f'count_cols_df_OHLCVA_downloaded: {count_cols_df_OHLCVA_downloaded}')
print(f'count_cols_df_adjOHLCV: {count_cols_df_adjOHLCV}')

count_symbols: 1319
count_df_close_cols: 1319
count_cols_df_OHLCVA_downloaded: 7914
count_cols_df_adjOHLCV: 6595


In [22]:
close_vs_Yahoo(test_symbols, df_adjOHLCV)

symbol:     A   Date: 2022-03-04   df_Close:  133.01483     Yahoo_Close:  133.01483   %_dif_Close: 0.00000
symbol:   LMT   Date: 2022-03-04   df_Close:  446.68860     Yahoo_Close:  446.68860   %_dif_Close: 0.00000
symbol:   SPY   Date: 2022-03-04   df_Close:  425.27069     Yahoo_Close:  425.27069   %_dif_Close: 0.00000
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [23]:
_sym = 'NOC'
s_start, s_end = -252, -248
df_adjOHLCV[_sym].iloc[s_start:s_end]
df_OHLCVA_downloaded[_sym].iloc[s_start:s_end]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-02,451.769989,457.540009,436.059998,453.390015,447.011108,1784400.0
2022-03-03,452.399994,455.089996,441.369995,449.690002,443.363159,1426100.0
2022-03-04,454.140015,472.940002,450.010010,468.570007,461.977478,1675700.0
2022-03-07,472.420013,490.820007,471.950012,477.950012,471.225555,2401800.0


In [24]:
df_adjOHLCV.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-02-24  141.460007  142.240005  138.720001  141.199997  1522100.0   
2023-02-27  142.880005  143.759995  141.809998  142.179993  1149600.0   
2023-02-28  141.750000  143.380005  141.389999  141.970001  2342500.0   
2023-03-01  138.580002  139.990005  136.250000  137.509995  3132800.0   
2023-03-02  136.300003  141.690002  136.199997  141.669998  1879178.0   

                   AA                                            ...  \
                 Open       High        Low      Close   Volume  ...   
Date                                                             ...   
2023-02-24  46.930000  46.930000  45.270000  46.450001  4587700  ...   
2023-02-27  46.849998  48.435001  46.369999  48.040001  4931200  ...   
2023-02-28  48.250000  50.040001  47.799999  48.939999  3871400  ...   
2023-03-01  51.290001  52.959999  50.529999  51.650002  7599600  ...   
2023-03-02  50.189999  53.709999  50.060001  53.689999  6469643  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-02-24  24.160000  24.469999  23.469999  23.700001  617400.0  22.610001   
2023-02-27  24.000000  24.040001  23.200001  23.200001  258500.0  23.280001   
2023-02-28  23.280001  23.650000  23.070000  23.260000  356100.0  23.100000   
2023-03-01  22.990000  23.660000  22.930000  23.209999  348400.0  22.870001   
2023-03-02  23.150000  23.549999  23.049999  23.400000  247910.0  22.129999   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-02-24  23.000000  22.490000  22.969999  1644200.0  
2023-02-27  23.379999  22.910000  23.010000  1066100.0  
2023-02-28  23.389999  22.990000  23.000000  1226700.0  
2023-03-01  22.910000  22.370001  22.400000  1323800.0  
2023-03-02  23.170000  21.980000  23.070000  1184935.0  

[5 rows x 6595 columns]

In [25]:
syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
_df = df_close[syms_def]
_df.tail()


,NOC,HII,AVAV,LMT,CW,HEI
Date,,,,,,
2023-02-24,470.910004,220.750000,85.129997,477.394318,177.520004,174.869995
2023-02-27,470.070007,217.380005,85.599998,476.489990,176.679993,174.669998
2023-02-28,464.109985,215.199997,85.739998,474.260010,174.789993,165.570007
2023-03-01,462.549988,215.729996,84.760002,474.329987,176.009995,171.410004
2023-03-02,468.269989,217.520004,86.870003,478.309998,178.779999,174.399994


In [26]:
syms_LNG = ['LNG', 'CHK', 'GLNG']
_df = df_close[syms_LNG]
_df.tail()

,LNG,CHK,GLNG
Date,,,
2023-02-24,157.770004,85.400002,23.010000
2023-02-27,158.500000,81.930000,22.620001
2023-02-28,157.339996,80.809998,22.830000
2023-03-01,160.789993,81.339996,22.730000
2023-03-02,163.320007,82.080002,22.540001


In [27]:
_df = df_close[symbols_add]
_df.tail()

,SPY,VGT,FTEC,QQQ,VWO,VTV,BNDX,USO,JJC,BCI,NG
Date,,,,,,,,,,,
2023-02-24,396.380005,350.920013,103.720001,291.850006,39.459999,140.309998,47.830837,67.099998,20.108000,20.690001,5.50
2023-02-27,397.730011,352.489990,104.269997,293.940002,39.660000,140.300003,47.720974,66.250000,20.320000,20.780001,5.54
2023-02-28,396.260010,352.119995,104.120003,293.559998,39.419998,139.610001,47.700996,67.209999,20.716000,20.780001,5.59
2023-03-01,394.739990,349.399994,103.370003,291.200012,40.209999,139.449997,47.540001,68.070000,21.209999,21.040001,5.75
2023-03-02,397.809998,353.549988,104.570000,293.609985,40.389999,140.160004,47.520000,68.300003,20.739901,20.940001,5.71


In [28]:
# syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
s_start, s_end = -252, -247
_df = df_close[test_symbols].iloc[s_start:s_end]
_df

,A,LMT,SPY
Date,,,
2022-03-02,132.289597,438.742493,430.899384
2022-03-03,136.273407,436.617065,428.754181
2022-03-04,133.014832,446.688599,425.270691
2022-03-07,129.488037,454.488495,412.734100
2022-03-08,128.474701,437.757721,409.604858
